hello


In [7]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Annual_Report_2023.pdf')
docs=loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 1000,  # size of each chunk created
    chunk_overlap  = 300,  # size of  overlap between chunks in order to maintain the context
    

)
chunks = r_splitter.split_documents(docs)

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [10]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunks,embeddings)  

In [11]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (


"""Generate a 3-4 page Risk Assessment Report based on the provided Annual Report, incorporating quantitative data, financial ratios, and key performance metrics wherever applicable. The report should be well-structured and include detailed analysis under the following headings:

1️ Company Overview
Provide a brief introduction to the company, including its industry, market presence, and core business operations.
Mention key financial highlights, such as:
Revenue, Net Profit, and Market Share
Compound Annual Growth Rate (CAGR) over the last 3-5 years
2️ Financial Risk
Analyze liquidity risks, using:
Current Ratio and Quick Ratio
Assess leverage and solvency risks, considering:
Debt-to-Equity Ratio and Interest Coverage Ratio
Provide a 3-year trend analysis of these ratios and justify the findings.
Examine profitability risks based on:
Net Profit Margin, Return on Equity (ROE), Return on Assets (ROA), and Earnings Per Share (EPS) trends over multiple years.
Display actual numbers and explain the implications of the trends.
Identify potential risks related to cash flow volatility and debt repayment capacity.
3️ Operational Risk
Identify risks in:
Supply chain disruptions, production, and logistics.
IT, cybersecurity, or infrastructure-related vulnerabilities.
Assess HR-related risks, such as:
Labor strikes, high employee attrition, and skill shortages.
4️ Market Risk
Analyze industry trends, macroeconomic factors, and the competitive landscape.
Evaluate risks due to:
Market volatility, inflation, interest rates, and currency fluctuations.
Assess potential customer demand shifts and technological disruptions.
5️ Governance & Regulatory Risk
Identify risks related to:
Corporate governance, compliance issues, and legal proceedings.
Assess risks from:
Government policies, taxation changes, and international trade regulations.
Include Environmental, Social, and Governance (ESG) considerations and sustainability risks.
6️ Risk Rating & Conclusion
Assign an overall risk rating (High, Medium, or Low) based on the analysis.
Summarize key risk factors and provide recommendations for mitigating these risks.
 Note:
1 All financial ratios must be presented with actual numerical values over multiple years and properly justified.
2 Cover all the above sections comprehensively, ensuring a professional and analytical approach.
3 Use a data-driven methodology, linking ratios with risk insights."""
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [47]:
response = rag_chain.invoke({"input": """  What is the Net Profit Margin, ROE, ROA, and EPS trends over the past 3 years?
"""})
print(response["answer"])

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [13]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load PDF
loader = PyPDFLoader('Annual_Report_2023.pdf')
docs = loader.load()

# Split text into chunks
r_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=1000,
    chunk_overlap=300,
)
chunks = r_splitter.split_documents(docs)

# Create embeddings and store in FAISS
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(chunks, gemini_embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Define LLM
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3, max_tokens=500)

# Queries to extract financial metrics
financial_queries = [
    "What is the Revenue of the company over the past 3 years?",
    "What is the Net Profit of the company over the past 3 years?",
    "What is the Current Ratio and Quick Ratio of the company for the past 3 years?",
    "What is the Debt-to-Equity Ratio and Interest Coverage Ratio for the past 3 years?",
    "What is the Net Profit Margin, ROE, ROA, and EPS trends over the past 3 years?",
    "What are the major cash flow risks identified in the report?",
    "What are the key operational risks mentioned in the report?",
    "What are the major market risks affecting the company?",
    "What are the key governance and regulatory risks mentioned in the report?"
]

# Extract financial metrics and store them
def extract_metrics(queries, retriever, llm):
    extracted_data = {}
    for query in queries:
        response = retriever.invoke({"input": query})
        extracted_data[query] = response["answer"]
    return extracted_data

financial_data = extract_metrics(financial_queries, retriever, gemini_llm)

# Format extracted data as context
formatted_context = "\n".join([f"{key}: {value}" for key, value in financial_data.items()])

# Risk Assessment Report Prompt
risk_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Generate a 3-4 page Risk Assessment Report based on the provided Annual Report, incorporating quantitative data, 
    financial ratios, and key performance metrics wherever applicable. The report should include:
    
    1. **Company Overview**
    2. **Financial Risk** (Liquidity, Leverage, Profitability)
    3. **Operational Risk** (Supply Chain, IT, HR)
    4. **Market Risk** (Industry Trends, Inflation, Interest Rates)
    5. **Governance & Regulatory Risk**
    6. **Risk Rating & Conclusion**
    
    Ensure financial ratios and key insights from the extracted data are used.
    {context}
    """),
    ("human", "Generate the Risk Assessment Report based on the extracted data.")
])

# Create final Risk Report generation chain
risk_report_chain = create_stuff_documents_chain(gemini_llm, risk_assessment_prompt)
final_report = risk_report_chain.invoke({"context": formatted_context})

print(final_report["answer"])


TypeError: expected string or bytes-like object

In [50]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load PDF
loader = PyPDFLoader('Annual_Report_2023.pdf')
docs = loader.load()

# Split text into chunks
r_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=1000,
    chunk_overlap=300,
)
chunks = r_splitter.split_documents(docs)

# Create embeddings and store in FAISS
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(chunks, gemini_embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Define LLM
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3, max_tokens=500)

# Queries to extract financial metrics
financial_queries = [
    "What is the Revenue of the company over the past 3 years?",
    "What is the Net Profit of the company over the past 3 years?",
    "What is the Current Ratio and Quick Ratio of the company for the past 3 years?",
    "What is the Debt-to-Equity Ratio and Interest Coverage Ratio for the past 3 years?",
    "What is the Net Profit Margin, ROE, ROA, and EPS trends over the past 3 years?",
    "What are the major cash flow risks identified in the report?",
    "What are the key operational risks mentioned in the report?",
    "What are the major market risks affecting the company?",
    "What are the key governance and regulatory risks mentioned in the report?"
]

# Function to extract financial metrics
def extract_metrics(queries, retriever, llm):
    extracted_data = {}
    for query in queries:
        retrieved_docs = retriever.invoke(query)  # Get relevant document chunks
        context = "\n".join([doc.page_content for doc in retrieved_docs])  # Extract text
        
        # Ask LLM to summarize extracted financial data
        response = llm.invoke({"input": f"{context}\n\nBased on the above information, answer: {query}"})
        extracted_data[query] = response["answer"]
    
    return extracted_data

# Extract financial data
financial_data = extract_metrics(financial_queries, retriever, gemini_llm)

# Format extracted data as context
formatted_context = "\n".join([f"{key}: {value}" for key, value in financial_data.items()])

# Risk Assessment Report Prompt
risk_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Generate a 3-4 page Risk Assessment Report based on the provided Annual Report, incorporating quantitative data, 
    financial ratios, and key performance metrics wherever applicable. The report should include:
    
    1. **Company Overview**
    2. **Financial Risk** (Liquidity, Leverage, Profitability)
    3. **Operational Risk** (Supply Chain, IT, HR)
    4. **Market Risk** (Industry Trends, Inflation, Interest Rates)
    5. **Governance & Regulatory Risk**
    6. **Risk Rating & Conclusion**
    
    Ensure financial ratios and key insights from the extracted data are used.
    {context}
    """),
    ("human", "Generate the Risk Assessment Report based on the extracted data.")
])

# Create final Risk Report generation chain
risk_report_chain = create_stuff_documents_chain(gemini_llm, risk_assessment_prompt)
final_report = risk_report_chain.invoke({"context": formatted_context})

# Print final Risk Assessment Report
print(final_report["answer"])


ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load and process PDF
loader = PyPDFLoader("Annual_Report_2023.pdf")
docs = loader.load()

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=1000,
    chunk_overlap=300,
)
chunks = splitter.split_documents(docs)

# Create embeddings and FAISS vector store
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(chunks, embedding_model)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Define LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3, max_tokens=500)

# Define financial queries
financial_queries = [
    "What is the Revenue of the company over the past 3 years?",
    "What is the Net Profit of the company over the past 3 years?",
    "What is the Current Ratio and Quick Ratio of the company for the past 3 years?",
    "What is the Debt-to-Equity Ratio and Interest Coverage Ratio for the past 3 years?",
    "What is the Net Profit Margin, ROE, ROA, and EPS trends over the past 3 years?",
    "What are the major cash flow risks identified in the report?",
    "What are the key operational risks mentioned in the report?",
    "What are the major market risks affecting the company?",
    "What are the key governance and regulatory risks mentioned in the report?"
]

# Function to extract financial metrics
def extract_metrics(queries, retriever, llm):
    extracted_data = {}
    for query in queries:
        retrieved_docs = retriever.invoke(query)  # Retrieve relevant chunks
        context = "\n".join([doc.page_content for doc in retrieved_docs])  # Extract text
        
        # Ask LLM to summarize financial data
        response = llm.invoke(f"{context}\n\nBased on the above information, answer: {query}")
        extracted_data[query] = response  # Assuming response is a string
    
    return extracted_data

# Extract financial data
financial_data = extract_metrics(financial_queries, retriever, llm)

# Format extracted data as context
formatted_context = "\n".join([f"{key}: {value}" for key, value in financial_data.items()])

# Risk Assessment Report Prompt
risk_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Generate a 3-4 page Risk Assessment Report based on the provided Annual Report, incorporating quantitative data, 
    financial ratios, and key performance metrics wherever applicable. The report should include:
    
    1. **Company Overview**
    2. **Financial Risk** (Liquidity, Leverage, Profitability)
    3. **Operational Risk** (Supply Chain, IT, HR)
    4. **Market Risk** (Industry Trends, Inflation, Interest Rates)
    5. **Governance & Regulatory Risk**
    6. **Risk Rating & Conclusion**
    
    Ensure financial ratios and key insights from the extracted data are used.
    {context}
    """),
    ("human", "Generate the Risk Assessment Report based on the extracted data.")
])

# Generate Risk Assessment Report
risk_report_chain = create_stuff_documents_chain(llm, risk_assessment_prompt)
final_report = risk_report_chain.invoke({"context": formatted_context})

# Print final Risk Assessment Report
print(final_report)


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [5]:
import os
import google.generativeai as genai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

# Configure API Key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Load and process PDF
loader = PyPDFLoader("Annual_Report_2023.pdf")
docs = loader.load()

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=1000,
    chunk_overlap=300,
)
chunks = splitter.split_documents(docs)

# Create embeddings and FAISS vector store
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(chunks, embedding_model)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Define financial queries
financial_queries = [
    "What is the Revenue of the company over the past 3 years?",
    "What is the Net Profit of the company over the past 3 years?",
    "What is the Current Ratio and Quick Ratio of the company for the past 3 years?",
    "What is the Debt-to-Equity Ratio and Interest Coverage Ratio for the past 3 years?",
    "What is the Net Profit Margin, ROE, ROA, and EPS trends over the past 3 years?",
    "What are the major cash flow risks identified in the report?",
    "What are the key operational risks mentioned in the report?",
    "What are the major market risks affecting the company?",
    "What are the key governance and regulatory risks mentioned in the report?"
]

# Initialize Gemini Pro Model
model = genai.GenerativeModel("gemini-pro")

# Function to extract financial metrics
def extract_metrics(queries, retriever, model):
    extracted_data = {}
    for query in queries:
        retrieved_docs = retriever.invoke(query)  # Retrieve relevant chunks
        context = "\n".join([doc.page_content for doc in retrieved_docs])  # Extract text
        
        # Ask Gemini Pro to summarize financial data
        prompt = f"{context}\n\nBased on the above information, answer: {query}"
        response = model.generate_content(prompt)
        
        if hasattr(response, "text"):
            extracted_data[query] = response.text  # Extract text
        else:
            extracted_data[query] = "Error retrieving response."
    
    return extracted_data

# Extract financial data
financial_data = extract_metrics(financial_queries, retriever, model)

# Format extracted data as context
formatted_context = "\n".join([f"{key}: {value}" for key, value in financial_data.items()])

# Risk Assessment Report Prompt
risk_assessment_prompt = f"""
Generate a 3-4 page Risk Assessment Report based on the provided Annual Report, incorporating quantitative data, 
financial ratios, and key performance metrics wherever applicable. The report should include:

1. **Company Overview**
2. **Financial Risk** (Liquidity, Leverage, Profitability)
3. **Operational Risk** (Supply Chain, IT, HR)
4. **Market Risk** (Industry Trends, Inflation, Interest Rates)
5. **Governance & Regulatory Risk**
6. **Risk Rating & Conclusion**

Ensure financial ratios and key insights from the extracted data are used.

{formatted_context}
"""

# Generate Risk Assessment Report
report_response = model.generate_content(risk_assessment_prompt)

# Print final Risk Assessment Report
if hasattr(report_response, "text"):
    print(report_response.text)
else:
    print("Error generating report.")


InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [6]:
import os
import google.generativeai as genai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

# Configure API Key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Load and process PDF
loader = PyPDFLoader("Annual_Report_2023.pdf")
docs = loader.load()

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "],
    chunk_size=1000,
    chunk_overlap=300,
)
chunks = splitter.split_documents(docs)

# Create embeddings and FAISS vector store
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db = FAISS.from_documents(chunks, embedding_model)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Define financial queries
financial_queries = [
    "What is the Revenue of the company over the past 3 years?",
    "What is the Net Profit of the company over the past 3 years?",
    "What is the Current Ratio and Quick Ratio of the company for the past 3 years?",
    "What is the Debt-to-Equity Ratio and Interest Coverage Ratio for the past 3 years?",
    "What is the Net Profit Margin, ROE, ROA, and EPS trends over the past 3 years?",
    "What are the major cash flow risks identified in the report?",
    "What are the key operational risks mentioned in the report?",
    "What are the major market risks affecting the company?",
    "What are the key governance and regulatory risks mentioned in the report?"
]

# Initialize Gemini 1.5 Pro Model
model = genai.GenerativeModel("gemini-1.5-pro")

# Function to extract financial metrics
def extract_metrics(queries, retriever, model):
    extracted_data = {}
    for query in queries:
        retrieved_docs = retriever.invoke(query)  # Retrieve relevant chunks
        context = "\n".join([doc.page_content for doc in retrieved_docs])  # Extract text
        
        # Ask Gemini 1.5 Pro to summarize financial data
        prompt = f"{context}\n\nBased on the above information, answer: {query}"
        response = model.generate_content(prompt)
        
        extracted_data[query] = response.text if hasattr(response, "text") else "Error retrieving response."
    
    return extracted_data

# Extract financial data
financial_data = extract_metrics(financial_queries, retriever, model)

# Format extracted data as context
formatted_context = "\n".join([f"{key}: {value}" for key, value in financial_data.items()])

# Risk Assessment Report Prompt
risk_assessment_prompt = f"""
Generate a 3-4 page Risk Assessment Report based on the provided Annual Report, incorporating quantitative data, 
financial ratios, and key performance metrics wherever applicable. The report should include:

1. **Company Overview**
2. **Financial Risk** (Liquidity, Leverage, Profitability)
3. **Operational Risk** (Supply Chain, IT, HR)
4. **Market Risk** (Industry Trends, Inflation, Interest Rates)
5. **Governance & Regulatory Risk**
6. **Risk Rating & Conclusion**

Ensure financial ratios and key insights from the extracted data are used.

{formatted_context}
"""

# Generate Risk Assessment Report
report_response = model.generate_content(risk_assessment_prompt)

# Print final Risk Assessment Report
print(report_response.text if hasattr(report_response, "text") else "Error generating report.")


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).